Connet to Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/Colab\ Notebooks

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks


Imports

In [0]:
%cd /content/gdrive/My\ Drive/Colab\ Notebooks
import os
import numpy as np
import glob
import csv
import keras
import scipy
from MRNet.partition import partition
from MRNet.data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib.pyplot import imshow
import random
import tensorflow as tf
from numpy.random import seed
from sklearn.utils import class_weight
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import auc as roc_auc
from MRNet.AlexNet_model_generator import MRNet_AlexNet_model

/content/gdrive/My Drive/Colab Notebooks


Training

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

%cd /content/gdrive/My\ Drive/Colab\ Notebooks
import os
import numpy as np
import glob
import csv
import keras
import scipy
from MRNet.partition import partition
from MRNet.data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib.pyplot import imshow
import random
import tensorflow as tf
from numpy.random import seed
from sklearn.utils import class_weight
from MRNet.AlexNet_model_generator import MRNet_AlexNet_model

label_type = 'abnormal'
exam_view = 'axial'
model, cbs = MRNet_AlexNet_model(1, 0.00001, [label_type, exam_view])
print("model output shape: ",model(tf.ones((2,51, 227, 227, 3))).shape)
model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_AlexNet/"+label_type+"/"+exam_view+"/weights.12.hdf5")
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/"
p, l = partition(data_path)

pos = 0
for e in p['train'][exam_view]:
  if l[e]["abnormal"] == 1:
    pos += 1
total = len(p['train'][exam_view])
neg = total - pos
class_count_train = {0: neg, 1: pos}
pos = 0
for e in p['test'][exam_view]:
  if l[e]["abnormal"] == 1:
    pos += 1
total = len(p['test'][exam_view])
neg = total - pos
class_count_test = {0: neg, 1: pos}
class_weight_test = {0: total/neg, 1: total/pos}

batch_size = 1
train_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_train , batch_size=1,model="AlexNet",data_type='train', label_type=label_type, exam_type=exam_view, scale_to=(227,227), aug_size=0)
print("train_data_gen: ", train_data_gen)
test_data_gen = MRNet_data_generator(data_path, p, l, class_count=class_count_test, batch_size=1,model="AlexNet",data_type='test', label_type=label_type, exam_type=exam_view, scale_to=(227,227), aug_size=0)
print("test_data_gen: ", test_data_gen)
model.fit(x=train_data_gen,validation_data=test_data_gen, epochs=12, verbose=True, callbacks=cbs,shuffle=False, use_multiprocessing=True, workers=6)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
checkpoint_dir: /content/gdrive/My Drive/Colab Notebooks/MRNet/training_AlexNet/abnormal/axial/
model output shape:  (1, 1)
Initializing Data Generator:
factor :  4.242268041237113
model:  AlexNet
data type:  train
Combination:  abnormal  and  axial
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/train
Number of inputs:  1646
input size:  (227, 227)
train_data_gen:  <MRNet.data_generator.MRNet_data_generator object at 0x7ff1a0089710>
Initializing Data Generator:
factor :  3.9130434782608696
model:  AlexNet
data type:  test
Combination:  abnormal  and  axial
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/test
Number of inputs:  180
input size:  (227, 227)
test_data_gen:  <MRNet.data_generator.MRNet_data_generator object at 0x7ff1a038d0b8>
Epoch 1/12
1646/1646 [==============================] - 253s 154ms/step - loss: 0.0667 - tp: 401.8870 - fp: 4.

Validation/Testing

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

%cd /content/gdrive/My\ Drive/Colab\ Notebooks
import os
import numpy as np
import glob
import csv
import keras
import scipy
from MRNet.partition import partition
from MRNet.data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib.pyplot import imshow
import random
import tensorflow as tf
from numpy.random import seed
from sklearn.utils import class_weight
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import auc as roc_auc
from MRNet.AlexNet_model_generator import MRNet_AlexNet_model

# evaluation

data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
combinations = [
                ["abnormal", "axial"],
                ["abnormal", "coronal"],
                ["abnormal", "sagittal"],
                ["ACL", "axial"],
                ["ACL", "coronal"],
                ["ACL", "sagittal"],
                ["meniscus", "axial"],
                ["meniscus", "coronal"],
                ["meniscus", "sagittal"],
]

for c in combinations:
  label_type = c[0]
  exam_view = c[1]
  model, cbs = MRNet_AlexNet_model(1, 0.00001,combination=[label_type, exam_view])
  model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_AlexNet/"+label_type+"/"+exam_view+"/weights.12.hdf5")
  p, l = partition(data_path)
  pos = 0
  for e in p['test'][exam_view]:
    if l[e][label_type] == 1:
      pos += 1
  total = len(p['test'][exam_view])
  neg = total - pos
  class_count_valid = {0: neg, 1: pos}
  batch_size = 1
  print(model.metrics_names)

  valid_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_valid , batch_size=1,model="AlexNet",data_type='test', shuffle=False,label_type=label_type, exam_type=exam_view, scale_to=(227,227), aug_size=0)
  d = model.evaluate_generator(valid_data_gen, total, verbose=1)
  print(model.metrics_names[0], "=",d[0],model.metrics_names[1], "=",d[1],model.metrics_names[2], "=",d[2],model.metrics_names[3], "=",d[3],model.metrics_names[4], "=",d[4],model.metrics_names[5], "=",d[5],model.metrics_names[6], "=",d[6],model.metrics_names[7], "=",d[7],model.metrics_names[8], "=",d[8]);
  model.predict_generator(valid_data_gen, 5, verbose=1)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks
checkpoint_dir: /content/gdrive/My Drive/Colab Notebooks/MRNet/training_AlexNet/abnormal/axial/
['loss', 'tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 'recall', 'auc']
Initializing Data Generator:
factor :  3.9130434782608696
model:  AlexNet
data type:  test
Combination:  abnormal  and  axial
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/test
Number of inputs:  180
input size:  (227, 227)
113/113 [==============================] - 8s 69ms/step
loss = 0.0001587993756402284 tp = 40.69911575317383 fp = 8.849557876586914 tn = 3.920353889465332 fn = 3.530973434448242 accuracy = 0.7728577256202698 precision = 0.801662027835846 recall = 0.931233823299408 auc = 0.7387261986732483
5/5 [==============================] - 0s 88ms/step
checkpoint_dir: /content/gdrive/My Drive/Colab Notebooks/MRN

Create the test set

In [0]:
import os
import pickle

data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"

with open(data_path+"/validation_exams.txt", "rb") as fp:
  v_set = pickle.load(fp)
os.mkdir(data_path+'/test/')
os.mkdir(data_path+'/test/'+'axial/')
os.mkdir(data_path+'/test/'+'coronal/')
os.mkdir(data_path+'/test/'+'sagittal/')

for e in v_set:
  os.rename(data_path + '/train/'+'axial/' + e + '.npy', data_path+'/test/'+'axial/' + e + '.npy')
  os.rename(data_path + '/train/'+'coronal/' + e + '.npy', data_path+'/test/'+'coronal/' + e + '.npy')
  os.rename(data_path + '/train/'+'sagittal/' + e + '.npy', data_path+'/test/'+'sagittal/' + e + '.npy')

figuring out ImageNet AlexNet Weights structure

In [0]:
import h5py

one = h5py.File("/content/gdrive/My Drive/Colab Notebooks/MRNet/alexnet_weights.h5",'r')
two = h5py.File("/content/gdrive/My Drive/Colab Notebooks/MRNet/MRNet-v1.0training_AlexNet/abnormal/axial/weights.15.hdf5",'r')

print(list(one.keys()))
print("===============================================")
# list(two.keys())

# x = two['alex_net_layer_1']

layer = Conv2D(filters=96, input_shape=(227,227), kernel_size=(11,11), strides=(4,4), padding="valid", activation = "relu")
layer.set_weights(one['conv_1']['conv_1_W'])
layer.get_weights()
# layer.add_weights(one['conv_1']['conv_1_W'])


# x = one['conv_2_2']

# conv1_weights = one['conv_1']['conv_1_W']
# max_pooling1_weights = one['conv_1']['conv_1_b']

# conv2_weights = one['conv_2_1']['conv_2_1_W'] + one['conv_2_2']['conv_2_2_W']
# max_pooling2_weights = one['conv_2_1']['conv_2_1_b'] + one['conv_2_2']['conv_2_2_b']

# conv3_weights = one['conv_3']['conv_3_W']

# conv4_weights = one['conv_4_1']['conv_4_1_W'] + one['conv_4_2']['conv_4_2_W']

# conv5_weights = one['conv_5_1']['conv_5_1_W'] + one['conv_5_2']['conv_5_2_W']
# max_pooling5_weights = one['conv_5_1']['conv_5_1_b'] + one['conv_5_2']['conv_5_2_b']



# print(list(x.keys()))
# print("===============================================")


# for z in one.keys():
#   print(z)
#   print("---------")
#   for y in one[z].keys():
#     print(y)
#     print(one[z][y])
#   print("=============")
# z = x['conv_1_W']
# for y in z:
#   print(y)